## Deliverable 1. Retrieve Weather Data.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests as req
import os
import gmaps
import json
from citipy import citipy
from datetime import datetime
from config import weather_api_key
from config import g_key
from scipy.stats import linregress
import time

In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?" + "units=Imperial" + "&APPID=" + weather_api_key

# Create zipped lat_lngs[] with 2000 random lat & lon values

In [3]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

# create coordinates[] from zipped lat_lngs[]

In [4]:
coordinates = list(lat_lngs)

# Use citipy to find nearest city for coordinates[] and append to cities[]

In [5]:
cities = []
for coordinate in coordinates:
    x = coordinate[0]
    y = coordinate[1]
    city_name    = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    country_code = citipy.nearest_city(coordinate[0], coordinate[1]).country_code
    #print(f"{x} {y} {city_name} {country_code}")
    if city_name not in cities:
        cities.append(city_name)
print(len(cities))
cities

761


['ozgon',
 'mto wa mbu',
 'ahipara',
 'yulara',
 'faanui',
 'tabiauea',
 'hobyo',
 'hithadhoo',
 'praia',
 'punta arenas',
 'port elizabeth',
 'severo-kurilsk',
 'ushuaia',
 'illoqqortoormiut',
 'itarema',
 'hermanus',
 'cape town',
 'maumere',
 'saint-philippe',
 'albany',
 'hobart',
 'taolanaro',
 'hasaki',
 'homer',
 'amderma',
 'qinzhou',
 'vila',
 'puerto ayora',
 'soligalich',
 'cayenne',
 'sakakah',
 'port alfred',
 'alekseyevsk',
 'rikitea',
 'narsaq',
 'zheleznodorozhnyy',
 'labuhan',
 'georgetown',
 'salym',
 'atuona',
 'avarua',
 'celestun',
 'moissala',
 'shawville',
 'bluff',
 'saldanha',
 'sinkat',
 'upernavik',
 'filadelfia',
 'palaiokhora',
 'meulaboh',
 'fortuna',
 'thessalon',
 'shimoda',
 'omutninsk',
 'ha giang',
 'qaanaaq',
 'mangochi',
 'castro',
 'jamestown',
 'mount gambier',
 'margate',
 'busselton',
 'chanute',
 'shaunavon',
 'ribeira grande',
 'carnarvon',
 'rincon de romos',
 'northam',
 'maua',
 'huarmey',
 'sinop',
 'okakarara',
 'port hedland',
 'new norf

# Function to populate citi_data[] from cities[] using OpenWeatherMap API

In [6]:
city_data = []

def get_cities_from_weatherfoo():
    print("Beginning Data Retrieval     ")
    print("-----------------------------")
    record_count = 1
    set_count = 1
    for i, city in enumerate(cities):
        # Group cities in sets of 50 for logging purposes.
        if (i % 50 == 0 and i >= 50):
            set_count += 1
            record_count = 1
        # Create endpoint URL with each city.
        city_url = url + "&q=" + city.replace(" ","+")
        #print(city_url)
        # Log the URL, record, and set numbers and the city.
        print(f"Processing Record {record_count} of Set {set_count} | {city}")
        # Add 1 to the record count.
        record_count += 1
        # Run an API request for each of the cities.
        try:
            # Parse the JSON and retrieve data.
            #time.sleep(1)
            city_weather = req.get(city_url)
            #print(city_weather)
            city_weather_json = city_weather.json()
            #print(city_weather_json)
            #print(json.dumps(city_weather_json, indent=4, sort_keys=True))
            # Parse out the needed data.
            city_lat = city_weather_json["coord"]["lat"]
            city_lng = city_weather_json["coord"]["lon"]
            city_max_temp = city_weather_json["main"]["temp_max"]
            city_humidity = city_weather_json["main"]["humidity"]
            city_clouds = city_weather_json["clouds"]["all"]
            city_wind = city_weather_json["wind"]["speed"]
            city_weather_desc = city_weather_json["weather"][0]["description"]
            city_country = city_weather_json["sys"]["country"]
            # Convert the date to ISO standard.
            city_date = datetime.utcfromtimestamp(city_weather_json["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            # Append the city information into city_data list.
            city_data.append({"City": city,
                              "Country": city_country,
                              "Lat": city_lat,
                              "Lng": city_lng,
                              "Max Temp": city_max_temp,
                              "Humidity": city_humidity,
                              "Cloudiness": city_clouds,
                              "Wind Speed": city_wind,
                              "Description": city_weather_desc})
 
        # If an error is experienced, skip the city.
        except:
            print("City not found. Skipping...")
            pass

    print("-----------------------------")
    print("Data Retrieval Complete      ")
    print("-----------------------------")


# Create city_data_df DataFrame and save WeatherPy_Database.csv

In [7]:
if os.path.isfile("WeatherPy_Database.csv"):
    # if csv file exists, read it
    city_data_df = pd.read_csv("WeatherPy_Database.csv")
else:
    # else create it
    get_cities_from_weatherfoo()
    city_data_df = pd.DataFrame(city_data)
    city_data_df.to_csv("WeatherPy_Database.csv")
    
city_data_df.head(10)

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ozgon
City not found. Skipping...
Processing Record 2 of Set 1 | mto wa mbu
Processing Record 3 of Set 1 | ahipara
Processing Record 4 of Set 1 | yulara
Processing Record 5 of Set 1 | faanui
Processing Record 6 of Set 1 | tabiauea
City not found. Skipping...
Processing Record 7 of Set 1 | hobyo
Processing Record 8 of Set 1 | hithadhoo
Processing Record 9 of Set 1 | praia
Processing Record 10 of Set 1 | punta arenas
Processing Record 11 of Set 1 | port elizabeth
Processing Record 12 of Set 1 | severo-kurilsk
Processing Record 13 of Set 1 | ushuaia
Processing Record 14 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 15 of Set 1 | itarema
Processing Record 16 of Set 1 | hermanus
Processing Record 17 of Set 1 | cape town
Processing Record 18 of Set 1 | maumere
Processing Record 19 of Set 1 | saint-philippe
Processing Record 20 of Set 1 | albany
Processing Record 21 of Set 1 

Processing Record 37 of Set 4 | esperance
Processing Record 38 of Set 4 | norman wells
Processing Record 39 of Set 4 | salalah
Processing Record 40 of Set 4 | thunder bay
Processing Record 41 of Set 4 | tura
Processing Record 42 of Set 4 | sovetskaya
Processing Record 43 of Set 4 | tayibe
City not found. Skipping...
Processing Record 44 of Set 4 | lithakia
Processing Record 45 of Set 4 | kangaatsiaq
Processing Record 46 of Set 4 | codrington
Processing Record 47 of Set 4 | bikaner
Processing Record 48 of Set 4 | harper
Processing Record 49 of Set 4 | longyearbyen
Processing Record 50 of Set 4 | marzuq
Processing Record 1 of Set 5 | iqaluit
Processing Record 2 of Set 5 | vasterhaninge
Processing Record 3 of Set 5 | saint pete beach
Processing Record 4 of Set 5 | victor harbor
Processing Record 5 of Set 5 | san policarpo
Processing Record 6 of Set 5 | divnogorsk
Processing Record 7 of Set 5 | cap malheureux
Processing Record 8 of Set 5 | nagato
Processing Record 9 of Set 5 | douglas
Proc

Processing Record 25 of Set 8 | san quintin
Processing Record 26 of Set 8 | ca mau
Processing Record 27 of Set 8 | savona
Processing Record 28 of Set 8 | korla
Processing Record 29 of Set 8 | dikson
Processing Record 30 of Set 8 | biak
Processing Record 31 of Set 8 | mehamn
Processing Record 32 of Set 8 | terrace
Processing Record 33 of Set 8 | bima
Processing Record 34 of Set 8 | jiwani
Processing Record 35 of Set 8 | pochutla
Processing Record 36 of Set 8 | antofagasta
Processing Record 37 of Set 8 | gazanjyk
Processing Record 38 of Set 8 | wajir
Processing Record 39 of Set 8 | san cristobal
Processing Record 40 of Set 8 | gunjur
Processing Record 41 of Set 8 | nuuk
Processing Record 42 of Set 8 | provideniya
Processing Record 43 of Set 8 | saleaula
City not found. Skipping...
Processing Record 44 of Set 8 | likasi
Processing Record 45 of Set 8 | bara
Processing Record 46 of Set 8 | nelson bay
Processing Record 47 of Set 8 | diourbel
Processing Record 48 of Set 8 | shelburne
Processi

Processing Record 13 of Set 12 | tabuk
Processing Record 14 of Set 12 | ibra
Processing Record 15 of Set 12 | rognan
Processing Record 16 of Set 12 | haines junction
Processing Record 17 of Set 12 | vostok
Processing Record 18 of Set 12 | lovington
Processing Record 19 of Set 12 | sawakin
Processing Record 20 of Set 12 | tongliao
Processing Record 21 of Set 12 | ankazobe
Processing Record 22 of Set 12 | labelle
Processing Record 23 of Set 12 | dinguiraye
Processing Record 24 of Set 12 | gelemso
Processing Record 25 of Set 12 | pangnirtung
Processing Record 26 of Set 12 | venezuela
Processing Record 27 of Set 12 | aitape
Processing Record 28 of Set 12 | moussoro
Processing Record 29 of Set 12 | kawalu
Processing Record 30 of Set 12 | nurota
Processing Record 31 of Set 12 | naze
Processing Record 32 of Set 12 | jahrom
City not found. Skipping...
Processing Record 33 of Set 12 | zhitikara
City not found. Skipping...
Processing Record 34 of Set 12 | santa cruz
Processing Record 35 of Set 1

Processing Record 50 of Set 15 | nabire
Processing Record 1 of Set 16 | aldan
Processing Record 2 of Set 16 | boralday
Processing Record 3 of Set 16 | gulshat
City not found. Skipping...
Processing Record 4 of Set 16 | copiapo
Processing Record 5 of Set 16 | arima
Processing Record 6 of Set 16 | grindavik
Processing Record 7 of Set 16 | derzhavinsk
Processing Record 8 of Set 16 | malanje
Processing Record 9 of Set 16 | marsa matruh
Processing Record 10 of Set 16 | valparaiso
Processing Record 11 of Set 16 | rieti
-----------------------------
Data Retrieval Complete      
-----------------------------


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,mto wa mbu,TZ,-3.3500,35.8500,72.30,84,95,3.44,light rain
1,ahipara,NZ,-35.1667,173.1667,66.65,96,35,3.06,scattered clouds
2,yulara,AU,-25.2406,130.9889,82.40,32,5,11.50,clear sky
3,faanui,PF,-16.4833,-151.7500,82.13,72,100,8.81,overcast clouds
4,hobyo,SO,5.3505,48.5268,86.92,54,81,12.75,broken clouds
5,hithadhoo,MV,-0.6000,73.0833,83.08,71,18,9.84,few clouds
6,praia,CV,14.9215,-23.5087,71.60,53,0,17.27,clear sky
7,punta arenas,CL,-53.1500,-70.9167,46.40,66,0,13.80,clear sky
8,port elizabeth,ZA,-33.9180,25.5701,64.40,82,0,5.75,clear sky
9,severo-kurilsk,RU,50.6789,156.1250,31.01,94,100,18.48,light snow
